#### imports for the project

In [2]:
import nltk
from nltk.corpus import movie_reviews
import random
from prettytable import PrettyTable
import textwrap 
import numpy as np

#### Import excel with pandas Emotions

In [7]:
import pandas as pd
df_emotions = pd.read_excel('Diabetes-classification.xlsx', sheet_name ='Emotions')

# Preparing dataset
X = df_emotions.loc[:,'discussion_text']
y = df_emotions.loc[:,'Label']
x_text = open("Diabetes-classification.txt")
# removes all duplicates from list 
Labels = list(dict.fromkeys(y)) 

#### Multinominal NB classifer for Emotions

In [10]:
# the reviews will be stored as document pairs of words and category
#documents = np.dstack((y,X))


# the reviews will be stored as document pairs of words and category
documents = [(list(x_text.words(text)), emotion)
    for emotion in y
    for text in x_text]

#documents = [(text, emotions)
#    for emotions in y
#    for text in X ]


#give random order to the documents
random.shuffle(documents)

tab = PrettyTable(['Discussion text', 'Emotion'])
tab.horizontal_char = '-'

for (doc, cat) in documents[0:2]:
    feats = textwrap.fill(','.join(doc[:50]), width=40)
    tab.add_row([ feats, cat])
    tab.add_row([ '\n', '\n'])
    print(cat)

print(tab)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 5341: invalid start byte

#### loads in Patient Journey labels

In [ ]:
import pandas as pd
df_patient = pd.read_excel('Diabetes-classification.xlsx', sheet_name='Patient-journey') # Reads in excel
#print(df_patient.head(), df_patient.tail())